<a href="https://colab.research.google.com/github/richardGonzalez-std/InventarioKivyMVC/blob/main/tools/Build_APK_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIAM - Compilar APK en Google Colab

**Primera vez:** ~30 min (descarga SDK/NDK)

**Siguientes veces:** ~5-10 min (usa cache de Drive)

---

## 1. Configuracion

In [66]:
!rm -rf /content/drive/MyDrive/buildozer_cache/
# Ejecuta esto en una celda de Colab
!rm -rf .buildozer/android/platform/

In [67]:
# ============================================
# EDITA ESTAS LINEAS
# ============================================
GITHUB_REPO = "https://github.com/richardGonzalez-std/InventarioKivyMVC.git"
BRANCH = "main"

# Usar cache de buildozer en Drive (RECOMENDADO)
USE_CACHE = True

print(f"Repo: {GITHUB_REPO}")
print(f"Rama: {BRANCH}")
print(f"Cache: {'Activado' if USE_CACHE else 'Desactivado'}")

Repo: https://github.com/richardGonzalez-std/InventarioKivyMVC.git
Rama: main
Cache: Activado


## 2. Montar Google Drive

In [68]:
from google.colab import drive
drive.mount('/content/drive')
print('Drive montado')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive montado


## 3. Clonar repositorio

In [69]:
import os
import shutil
from pathlib import Path

WORK_DIR = '/content/InventarioKivyMVC'
CACHE_DIR = '/content/drive/MyDrive/buildozer_cache'

# Asegurar directorio base seguro
os.chdir('/content')

# Limpiar directorio de trabajo
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

# Clonar repo
!git clone --branch {BRANCH} --depth 1 {GITHUB_REPO} {WORK_DIR}

os.chdir(WORK_DIR)
print(f'\nProyecto clonado en: {WORK_DIR}')

Cloning into '/content/InventarioKivyMVC'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 46 (delta 0), reused 30 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 3.70 MiB | 22.56 MiB/s, done.

Proyecto clonado en: /content/InventarioKivyMVC


## 4. Restaurar cache de buildozer (si existe)

In [70]:
if USE_CACHE:
    buildozer_local = '/root/.buildozer'

    if os.path.exists(CACHE_DIR):
        print('Restaurando cache de buildozer desde Drive...')
        print('(Esto puede tomar unos minutos)')

        if os.path.exists(buildozer_local):
            shutil.rmtree(buildozer_local)

        shutil.copytree(CACHE_DIR, buildozer_local)

        # Calcular tamaño
        size_mb = sum(f.stat().st_size for f in Path(buildozer_local).rglob('*') if f.is_file()) / 1024 / 1024
        print(f'Cache restaurado: {size_mb:.0f} MB')
    else:
        print('No hay cache previo en Drive.')
        print('Primera compilacion sera lenta (~30 min)')
        print('El cache se guardara automaticamente al terminar.')
else:
    print('Cache desactivado')

No hay cache previo en Drive.
Primera compilacion sera lenta (~30 min)
El cache se guardara automaticamente al terminar.


## 5. Instalar dependencias

In [71]:
print('Instalando dependencias del sistema...')
!apt-get update -qq
!apt-get install -y -qq \
    python3-pip build-essential git python3-dev \
    ffmpeg libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev \
    libportmidi-dev libswscale-dev libavformat-dev libavcodec-dev zlib1g-dev \
    libgstreamer1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good \
    zip unzip openjdk-17-jdk autoconf libtool pkg-config \
    libncurses5-dev libncursesw5-dev cmake libffi-dev libssl-dev \
    gettext autopoint libltdl-dev libtool-bin lld m4
!apt-get install -y \
    python3-pip \
    build-essential \
    git \
    python3 \
    python3-dev \
    ffmpeg \
    libsdl2-dev \
    libsdl2-image-dev \
    libsdl2-mixer-dev \
    libsdl2-ttf-dev \
    libportmidi-dev \
    libswscale-dev \
    libavformat-dev \
    libavcodec-dev \
    zlib1g-dev \
    libgstreamer1.0 \
    gstreamer1.0-plugins-base \
    gstreamer1.0-plugins-good \
    libsqlite3-dev \
    sqlite3 \
    bzip2 \
    libbz2-dev \
    libssl-dev \
    openssl \
    libgdbm-dev \
    libgdbm-compat-dev \
    liblzma-dev \
    lzma \
    uuid-dev \
    libreadline-dev \
    libncursesw5-dev \
    libffi-dev \
    libdb-dev \
    expect

!pip install --upgrade pip -q
!pip install --upgrade buildozer cython==0.29.36 -q
print('Dependencias instaladas')

Instalando dependencias del sistema...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libgstreamer1.0-dev' for regex 'libgstreamer1.0'
Note, selecting 'libgstreamer1.0-0' for regex 'libgstreamer1.0'
build-essential is already the newest version (12.9ubuntu3).
bzip2 is already the newest version (1.0.8-5build1).
libbz2-dev is already the newest version (1.0.8-5build1).
libdb-dev is already the newest version (1:5.3.21~exp1ubuntu4).
libffi-dev is already the newest version (3.4.2-4).
libgdbm-compat-dev is already the newest version (1.23-1).
libgdbm-dev is already the newest version (1.23-1).
liblzma-dev is already the newest version (5.2.5-2ubuntu1).
libreadline-dev is already the newest version (8.1.2-1).
expect is already the newest 

## 6. Compilar APK

In [72]:
import time

os.chdir(WORK_DIR)

# Asegurar permisos de ejecución en SDK/NDK (pueden perderse al restaurar cache)
!chmod -R +x /root/.buildozer/android/platform/ 2>/dev/null || true

start = time.time()
!buildozer android clean
!buildozer -v android debug 2>&1 | tee build.log
elapsed = time.time() - start

print(f'\nTiempo de compilacion: {elapsed/60:.1f} minutos')

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Compiling '/content/InventarioKivyMVC/.buildozer/android/platform/build-armeabi-v7a_arm64-v8a/build/other_builds/python3/armeabi-v7a__ndk_target_21/python3/Lib/idlelib/idle_test/test_debugobj_r.py'...
Compiling '/content/InventarioKivyMVC/.buildozer/android/platform/build-armeabi-v7a_arm64-v8a/build/other_builds/python3/armeabi-v7a__ndk_target_21/python3/Lib/idlelib/idle_test/test_delegator.py'...
Compiling '/content/InventarioKivyMVC/.buildozer/android/platform/build-armeabi-v7a_arm64-v8a/build/other_builds/python3/armeabi-v7a__ndk_target_21/python3/Lib/idlelib/idle_test/test_editmenu.py'...
Compiling '/content/InventarioKivyMVC/.buildozer/android/platform/build-armeabi-v7a_arm64-v8a/build/other_builds/python3/armeabi-v7a__ndk_target_21/python3/Lib/idlelib/idle_test/test_editor.py'...
Compiling '/content/InventarioKivyMVC/.buildozer/android/platform/build-armeabi-v7a_arm64-v8a/build/other_builds/python3/armeabi-v7a__nd

## 8. Guardar APK en Drive

In [73]:
bin_path = Path(WORK_DIR) / 'bin'
apks = list(bin_path.glob('*.apk')) if bin_path.exists() else []

if apks:
    apk_path = apks[0]
    size_mb = apk_path.stat().st_size / 1024 / 1024
    print(f'APK: {apk_path.name}')
    print(f'Tamano: {size_mb:.1f} MB')

    output_dir = '/content/drive/MyDrive/APKs'
    os.makedirs(output_dir, exist_ok=True)
    dest = f'{output_dir}/{apk_path.name}'
    shutil.copy2(apk_path, dest)
    print(f'\nGuardado en: {dest}')
else:
    print('ERROR: No se genero el APK')
    print('\nErrores encontrados:')
    !grep -i "error" build.log | tail -20

APK: siam-0.1-armeabi-v7a_arm64-v8a-debug.apk
Tamano: 41.1 MB

Guardado en: /content/drive/MyDrive/APKs/siam-0.1-armeabi-v7a_arm64-v8a-debug.apk


## 9. Descargar APK

In [74]:
if apks:
    from google.colab import files
    files.download(str(apk_path))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>